In [1]:
import opensim as osim

In [2]:
import numpy as np
import math as m

import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd

### Import Transformed Markers

In [3]:
markers_df = pd.read_csv('model_update/tps_warping_results/markers_transformed.csv', index_col=0)
markers_df.head()

,body,location
name,,
ASIS_l,pelvis,[-1.78282254e-04 5.96425362e-05 -1.09615674e-01]
ASIS_r,pelvis,[ 1.78282254e-04 -5.96425362e-05 1.09615674e-01]
PSIS_l,pelvis,[-0.12944955 0.00051474 -0.04332435]
PSIS_r,pelvis,[-0.12918863 -0.00037401 0.0429037 ]
pelvis_origin,pelvis,[-4.26325641e-17 0.00000000e+00 2.21132140e-16]


In [4]:
def return_dataframe(markers_df, body):
    extract_df = markers_df[markers_df['body']== body]['location']
    ind = extract_df.index
    col = ['x', 'y', 'z']
    list_of_data = [row[1:-1].split() for row in extract_df]
    #print(list_of_data)
    convert = lambda x: 0 if x == '.' else float(x)
    data = np.array([[convert(x) for x in row] for row in list_of_data])
    data_df = pd.DataFrame(data, index=ind, columns=col)
    return data_df


In [5]:
pelvis_df = return_dataframe(markers_df, 'pelvis')
femur_r_df =  return_dataframe(markers_df, 'femur_r')
femur_l_df =  return_dataframe(markers_df, 'femur_l')
tibia_r_df =  return_dataframe(markers_df, 'tibia_r')
tibia_l_df =  return_dataframe(markers_df, 'tibia_l')
patella_r_df =  return_dataframe(markers_df, 'patella_r')
patella_l_df =  return_dataframe(markers_df, 'patella_l')
pelvis_df.tail()

,x,y,z
name,,,
isch_infer_l,-0.049100,-0.138270,-0.050037
pub_infer_c,-0.012608,-0.108552,0.000392
femur_r_center_in_pelvis,-0.039983,-0.079942,0.085153
femur_l_center_in_pelvis,-0.035456,-0.078876,-0.086802
torso_origin_in_pelvis,-0.087724,0.034880,-0.001546


### Parse current scaled model

In [6]:
tree=ET.parse("p03_scaled_2.osim")
root = tree.getroot()
# temp = mean_skin_markers_df*mean_skin_markers_df

#### Location of skin markers in child body frames

In [7]:
imported_markers_dict = {}
for marker in root.iter('Marker'):
    name = marker.attrib['name']
    body = marker.find('socket_parent_frame').text[9:]
    location = [float(val) for val in (marker.find('location').text).split()]
    imported_markers_dict[name] = [body] + location
    
imported_markers_df = pd.DataFrame.from_dict(imported_markers_dict).T
imported_markers_df = imported_markers_df.rename(columns={0:'body', 1:'x', 2:'y', 3:'z'})

In [8]:
imported_markers_df.index

Index(['C7', 'RBAK', 'CLAV', 'STRN', 'T10', 'RASI', 'RPSI', 'LPSI', 'LASI',
       'PE01', 'PE02', 'PE03', 'RTH1', 'RTH2', 'RTH3', 'RGT', 'RKNE', 'RMKNE',
       'LTH1', 'LTH2', 'LTH3', 'LGT', 'LKNE', 'LMKNE', 'RTB1', 'RTB2', 'RTB3',
       'RANK', 'RMMA', 'LTB1', 'LTB2', 'LTB3', 'LANK', 'LMMA', 'RHEE', 'RD5M',
       'RTOE', 'LHEE', 'LD5M', 'LTOE', 'pelvis_in_ground', 'femur_r_in_pelvis',
       'femur_l_in_pelvis', 'walker_knee_r_in_tibia_r', 'knee_r_in_femur_r',
       'patellofemoral_r_in_femur_r', 'walker_knee_l_in_tibia_l',
       'knee_l_in_femur_l', 'patellofemoral_l_in_femur_l',
       'ankle_r_tibia_r_offset', 'ankle_l_tibia_l_offset',
       'torso_origin_in_pelvis'],
      dtype='object')

In [9]:
mrkrs_torso = imported_markers_df[imported_markers_df['body'] == 'torso'][['x', 'y', 'z']]
mrkrs_pelvis = imported_markers_df[imported_markers_df['body'] == 'pelvis'][['x', 'y', 'z']]
mrkrs_femur_r = imported_markers_df[imported_markers_df['body'] == 'femur_r'][['x', 'y', 'z']]
mrkrs_femur_l = imported_markers_df[imported_markers_df['body'] == 'femur_l'][['x', 'y', 'z']]
mrkrs_tibia_r = imported_markers_df[imported_markers_df['body'] == 'tibia_r'][['x', 'y', 'z']]
mrkrs_tibia_l = imported_markers_df[imported_markers_df['body'] == 'tibia_l'][['x', 'y', 'z']]
mrkrs_foot_r = imported_markers_df.loc[['RHEE', 'RD5M','RTOE'], ['x', 'y', 'z']]
mrkrs_foot_l = imported_markers_df.loc[['LHEE', 'LD5M', 'LTOE'], ['x', 'y', 'z']]
                                         

In [10]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(width = 800, height = 600)
fig.add_trace(go.Scatter3d(x = pelvis_df['x'], y = pelvis_df['y'], z=pelvis_df['z'], mode = 'markers', name = 'pelvis_df'))
fig.add_trace(go.Scatter3d(x = mrkrs_pelvis['x'],  
                           y = mrkrs_pelvis['y'], 
                           z = mrkrs_pelvis['z'], mode = 'markers', name = 'osim skin mrkrs'))
fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

Skin markers and transformed muscles and markers fall essentially into the same set of axes. However, PE markers seem to indicate a significant lordosis...

### Translation of joint centres

In [11]:
# transfomed markers
femur_r_center_in_pelvis = pelvis_df.loc['femur_r_center_in_pelvis']
femur_l_center_in_pelvis = pelvis_df.loc['femur_l_center_in_pelvis']

knee_l_center_in_femur = femur_l_df.loc['knee_l_center_in_femur_l']
knee_r_center_in_femur = femur_r_df.loc['knee_r_center_in_femur_r']

patella_l_center_in_femur = femur_l_df.loc['patella_l_in_femur_l']
patella_r_center_in_femur = femur_r_df.loc['patella_r_in_femur_r']

tibia_l_center_in_tibia = tibia_l_df.loc['tibia_l_center']
tibia_r_center_in_tibia = tibia_r_df.loc['tibia_r_center']

talus_l_center_in_tibia = tibia_l_df.loc['talus_l_center_in_tibia']
talus_r_center_in_tibia = tibia_r_df.loc['talus_r_center_in_tibia']

torso_origin_in_pelvis = pelvis_df.loc['torso_origin_in_pelvis']

In [13]:
# markers in the current OSIM scaled model
old_femur_r_in_pelvis = mrkrs_pelvis.loc['femur_r_in_pelvis']
old_femur_l_in_pelvis = mrkrs_pelvis.loc['femur_l_in_pelvis']

old_knee_r_in_femur_r = mrkrs_femur_r.loc['knee_r_in_femur_r']
old_knee_l_in_femur_l = mrkrs_femur_l.loc['knee_l_in_femur_l']

old_patellofemoral_r_in_femur_r = mrkrs_femur_r.loc['patellofemoral_r_in_femur_r']
old_patellofemoral_l_in_femur_l = mrkrs_femur_l.loc['patellofemoral_l_in_femur_l']

old_walker_knee_r_in_tibia_r = mrkrs_tibia_r.loc['walker_knee_r_in_tibia_r']
old_walker_knee_l_in_tibia_l = mrkrs_tibia_l.loc['walker_knee_l_in_tibia_l']

old_ankle_r_tibia_r_offset = mrkrs_tibia_r.loc['ankle_r_tibia_r_offset']
old_ankle_l_tibia_l_offset = mrkrs_tibia_l.loc['ankle_l_tibia_l_offset']

old_torso_origin_in_pelvis = mrkrs_pelvis.loc['torso_origin_in_pelvis']

In [14]:
print('new', list(femur_r_center_in_pelvis), 'old', list(old_femur_r_in_pelvis) )

new [-0.03998291, -0.07994215, 0.0851525] old [-0.05191488049271365, -0.0768630395299027, 0.06709588850977007]


In [15]:
new_skin_mrkrs_femur_r = mrkrs_femur_r - (old_femur_r_in_pelvis - femur_r_center_in_pelvis)
new_skin_mrkrs_femur_l = mrkrs_femur_l - (old_femur_l_in_pelvis - femur_l_center_in_pelvis)
#new_skin_mrkrs_femur_r
#new_skin_mrkrs_femur_l

In [16]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(width = 800, height = 600)
fig.add_trace(go.Scatter3d(x = mrkrs_femur_r['x'], y = mrkrs_femur_r['y'], z=mrkrs_femur_r['z'], mode = 'markers', name = 'osim skin markers'))
fig.add_trace(go.Scatter3d(x = new_skin_mrkrs_femur_r['x'],  y = new_skin_mrkrs_femur_r['y'], z=new_skin_mrkrs_femur_r['z'], mode = 'markers', name = 'new_skin_mrkrs_femur_r mscl and mrkrs'))
fig.add_trace(go.Scatter3d(x = femur_r_df['x'],  y = femur_r_df['y'], z=femur_r_df['z'], mode = 'markers', marker=dict(size=4, opacity=0.8), name = 'transformed mrkrs'))

fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

In [17]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(width = 800, height = 600)
fig.add_trace(go.Scatter3d(x = mrkrs_femur_l['x'], 
                           y = mrkrs_femur_l['y'], 
                           z = mrkrs_femur_l['z'], mode = 'markers', name = 'osim skin markers'))
fig.add_trace(go.Scatter3d(x = new_skin_mrkrs_femur_l['x'],  y = new_skin_mrkrs_femur_l['y'], z=new_skin_mrkrs_femur_l['z'], mode = 'markers', name = 'new_skin_mrkrs_femur_r mscl and mrkrs'))
fig.add_trace(go.Scatter3d(x = femur_l_df['x'],  y = femur_l_df['y'], z=femur_l_df['z'], mode = 'markers', marker=dict(size=4, opacity=0.8), name = 'transformed mrkrs'))

fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

In [18]:
new_skin_mrkrs_tibia_r = mrkrs_tibia_r - (old_knee_r_in_femur_r - knee_r_center_in_femur)
new_skin_mrkrs_tibia_l = mrkrs_tibia_l - (old_knee_r_in_femur_r - knee_r_center_in_femur)

In [19]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(width = 800, height = 600)
fig.add_trace(go.Scatter3d(x = mrkrs_tibia_l['x'], 
                           y = mrkrs_tibia_l['y'], 
                           z = mrkrs_tibia_l['z'], mode = 'markers', name = 'osim skin markers'))
fig.add_trace(go.Scatter3d(x = new_skin_mrkrs_tibia_l['x'],  y = new_skin_mrkrs_tibia_l['y'], z=new_skin_mrkrs_tibia_l['z'], mode = 'markers', name = 'new_skin_mrkrs_femur_r mscl and mrkrs'))
fig.add_trace(go.Scatter3d(x = tibia_l_df['x'],  y = tibia_l_df['y'], z=tibia_l_df['z'], mode = 'markers', marker=dict(size=4, opacity=0.8), name = 'transformed mrkrs'))

fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

In [20]:
print(list(talus_r_center_in_tibia))
print(list(old_ankle_r_tibia_r_offset))

[-0.00261031, -0.39931207, -0.00217863]
[-0.0105542, -0.38169, 0.0]


In [21]:
new_skin_mrkrs_foot_r = mrkrs_foot_r - (old_ankle_r_tibia_r_offset - talus_r_center_in_tibia)
new_skin_mrkrs_foot_l = mrkrs_foot_l - (old_ankle_l_tibia_l_offset - talus_l_center_in_tibia)

In [22]:
new_skin_mrkrs_torso = mrkrs_torso - (old_torso_origin_in_pelvis - torso_origin_in_pelvis)

In [23]:
new_skin_mrkrs_pelvis = mrkrs_pelvis - 0.5 * (old_torso_origin_in_pelvis - torso_origin_in_pelvis)

In [24]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(width = 800, height = 600)
fig.add_trace(go.Scatter3d(x = pelvis_df['x'], y = pelvis_df['y'], z=pelvis_df['z'], mode = 'markers', name = 'pelvis_df'))
fig.add_trace(go.Scatter3d(x = mrkrs_pelvis['x'],  
                           y = mrkrs_pelvis['y'], 
                           z = mrkrs_pelvis['z'], mode = 'markers', name = 'osim skin mrkrs'))
fig.add_trace(go.Scatter3d(x = new_skin_mrkrs_pelvis['x'], y = new_skin_mrkrs_pelvis['y'], z=new_skin_mrkrs_pelvis['z'], mode = 'markers', name = 'new_skin_mrkrs_pelvis'))
fig.update_scenes(aspectmode="data" )
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()

## Parse TPS updated model

In [25]:
tree=ET.parse("p03_tps_transformed.osim")
root = tree.getroot()

In [26]:
new_skin_mrkrs_df = pd.concat([new_skin_mrkrs_torso, new_skin_mrkrs_pelvis, new_skin_mrkrs_femur_r, new_skin_mrkrs_femur_l, new_skin_mrkrs_tibia_r, new_skin_mrkrs_tibia_l,  new_skin_mrkrs_foot_r, new_skin_mrkrs_foot_l])

In [33]:
for marker in root.iter('Marker'):
    name = marker.attrib['name']
    # print(name)
    if name == 'pelvis_in_ground':
        pass
    else:
        old_text = marker.find('location').text
        new_text = f"{new_skin_mrkrs_df.loc[name, 'x']} {new_skin_mrkrs_df.loc[name, 'y']} {new_skin_mrkrs_df.loc[name, 'z']}"
        marker.find('location').text = new_text

In [34]:
tree.write('p03_tps_transformed_skin_updated.osim')

______________________________________________

### Import  skin markers static TRC

In [31]:
# import .trc file with experimental static marker positions
path_to_static = 'Session2/Static03/task.trc'
static_df = pd.read_csv(path_to_static, delimiter='\t', skiprows=3, header=[0,1], index_col=0)
static_df=static_df.drop('Time', axis = 1)

# adjust marker names
old_lst = list(static_df.columns)
new_lst = []
for i, value in enumerate(old_lst):
    if old_lst[i][0][:2] != 'Un':
        remember = value[0]
        new_lst.append(value)
    else:
        new_lst.append((remember, old_lst[i][1]))
static_df.columns = new_lst

# calculate a series of mean values for each makrer's coordinate
static_means = static_df.mean()[:-1]/1000

C:\Users\Ekatarina\AppData\Local\Temp\ipykernel_26984\3356606224.py:4: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



In [32]:
# create a dataframe from markers
key = ""
mean_dict = {}
for i, entry in enumerate(static_means.index):
    if entry[0] != key:
        key = entry[0]
        mean_dict[key] = [static_means[i]]
    else:
        mean_dict[key].append(static_means[i])
mean_skin_markers_df = (pd.DataFrame.from_dict(mean_dict)).T
mean_skin_markers_df = mean_skin_markers_df.rename(columns={0:'x', 1:'y', 2:'z'})
